In [2]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf
from numba import njit
import ta

In [81]:

# Data
symbol = 'SPXL'  # Using QQQ as an example

price = vbt.YFData.download(
    symbol,
    start='1/1/2023',
    interval='1d'
).get('Close')

fast_level = np.arange(9,27)
slow_level = np.arange(26,60)
EMA = vbt.IndicatorFactory.from_ta('EMAIndicator')
# ema13 = EMA.run(price, 5)
# ema48 = EMA.run(price, 10)

windows = np.arange(2, 101)
ema13, ema48 = EMA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])

entries = ema13.ema_indicator_crossed_above(ema48)
exits = ema48.ema_indicator_crossed_below(ema48)

@njit
def adjust_sl_func_nb(c):
    current_profit = (c.val_price_now - c.init_price) / c.init_price
    if current_profit >= 0.6:
        return 0.4, True
    if current_profit >= 0.40:
        return 0.05, True
    elif current_profit >= 0.2:
        return 0.01, True
    # elif current_profit >= 0.05:
    #     return 0.01, True
    # else:
    #     return 0.2, False
    return c.curr_stop, c.curr_trail

take_profit_pct = 20  # 10%
stop_loss_pct = 0.1  # -5%

# Backtest with vectorbt, specifying short entries and exits
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    short_entries=exits,
    short_exits=entries,
    # init_cash=10000,
    # size=1,
    # size_type=vbt.portfolio.enums.SizeType.Percent,
    sl_stop=stop_loss_pct,
    adjust_sl_func_nb=adjust_sl_func_nb,
    tp_stop=take_profit_pct,
    freq='1D'
)

In [84]:
pf.total_return()[::-1]

fast_window  slow_window
99           100            0.000000
98           100            0.000000
             99             0.000000
97           100            0.000000
             99             0.000000
                              ...   
2            7              0.577342
             6              0.577342
             5              0.566636
             4              0.616653
             3              0.834084
Name: total_return, Length: 4851, dtype: float64

In [82]:
xx = pf.get_orders()

# pf.cumulative_returns().vbt.plot().show()
max_pf = pf.total_return().idxmax()

opf = pf[max_pf]
opf.plot().show()

# Show performance metrics
print(max_pf)
print(opf.stats())


(2, 3)
Start                         2023-01-03 05:00:00+00:00
End                           2024-04-01 04:00:00+00:00
Period                                312 days 00:00:00
Start Value                                       100.0
End Value                                    183.408432
Total Return [%]                              83.408432
Benchmark Return [%]                           120.3787
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              32.791902
Max Drawdown Duration                 119 days 00:00:00
Total Trades                                          9
Total Closed Trades                                   8
Total Open Trades                                     1
Open Trade PnL                                -1.253303
Win Rate [%]                                       62.5
Best Trade [%]                                30.955156
Worst Trade [%]                          

In [80]:
pf.get_orders()

In [72]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side


In [54]:
xx.config['close'][max_pf]

Date
2018-01-02 05:00:00+00:00     43.365036
2018-01-03 05:00:00+00:00     44.132732
2018-01-04 05:00:00+00:00     44.689304
2018-01-05 05:00:00+00:00     45.533764
2018-01-08 05:00:00+00:00     45.812054
                                ...    
2024-03-25 04:00:00+00:00    131.880005
2024-03-26 04:00:00+00:00    130.809998
2024-03-27 04:00:00+00:00    134.080002
2024-03-28 04:00:00+00:00    134.029999
2024-04-01 04:00:00+00:00    133.193405
Name: (5, 10), Length: 1571, dtype: float64

In [73]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

AttributeError: 'numpy.float64' object has no attribute 'vbt'